NCF Recommender with Explict Feedback

In [1]:
#Context-Aware Neural Collaborative Filtering (CA-NCF)
#Author: Isam Mashhour Al Jawarneh
#Date: 30 January 2019

from zoo.pipeline.api.keras.layers import *
from zoo.models.recommendation import *
from zoo.models.recommendation.utils import *
from zoo.common.nncontext import init_nncontext
import os
import sys
import datetime as dt
from bigdl.dataset.transformer import *
from bigdl.dataset.base import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.optim import *
from bigdl.util.common import *
import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
sc = init_nncontext(create_spark_conf().setAppName("CA_NCF Tripadvisor"))

In [3]:
from scipy import stats
import numpy as np
import pandas as pd
#import researchpy as rp


In [4]:
#using pandas to read data from csv file

df = pd.read_csv("datasets/tripAdvisor/Data_TripAdvisor_v2.csv", names=['UserID', 'ItemID', 'rating','TripType'], header=None, usecols=[0,1,2,8])[['UserID', 'ItemID', 'rating','TripType']]


In [5]:
# for constructing objects containing items, the contexts and context conditions

class MyClass(object):
    def __init__(self, item,condition,context):
        self.number = number
        self.item = item
        self.condition = condition
        self.context = context

In [6]:
######ITEM SPLITTING Procedure############
## as described in :
## Experimental evaluation of context-dependent collaborative filtering using item splitting
## https://dl.acm.org/citation.cfm?id=2579997
## output a list (item, condition, context), where condition is the context condition that will be used for splitting the item

#profiling data
# some_list will contain items marked to be split as they show t-statistics greater than a threshold (4.0 in this case)

some_list = []
# we have only one context factor in this dataset: TripType
contextFactors = ['TripType']
dflist = df['ItemID'].unique().tolist()
cmax = 0.0
con = ''
context = ''
for i in dflist:
    cmax = 0.0
    for j in contextFactors:
        #print(j)
        #retreiving unique context condistions of a spicific context factor (we have only one context factor herein)
        conditionList = df[j].unique().tolist()
        #array_length1 = len(conditionList)
        #print(array_length1)
        for k in conditionList:
            #print(k)
            #splitting an item into two items based on the current context condition
            item1 = df[(df['ItemID'] == i) & (df[j] == k)]
            item1.reset_index(inplace= True)  
            item2 = df[(df['ItemID'] == i) & (df[j] != k)]
            item2.reset_index(inplace= True)
            l = stats.ttest_ind(item1['rating'], item2['rating'])
            #if l.statistic > 4.0:
            #print(i), print(l)
            if (l.statistic > cmax) :
                cmax = l.statistic
                con = k
                context = j
            #print(cmax)
        
    #print('finished')
    #HYPERPARAMETER cmax
    #considering for splitting only items that show ttest greater than a threshold (4.0 in this case)
    if cmax > 4.0:
        print('item ' + str(i) + ' cmax is ' + str(cmax) + ' con ' + str(con) + ' context ' + str(context))
        some_list.append(MyClass(str(i),str(con),str(context)))


/home/isam/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3506: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/isam/.local/lib/python3.5/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


item 503601 cmax is 4.225771273642583 con SOLO context TripType
item 95716 cmax is 4.800568148193977 con SOLO context TripType
item 88486 cmax is 4.024922359499622 con FAMILY context TripType
item 91503 cmax is 4.024922359499622 con SOLO context TripType
item 94389 cmax is inf con BUSINESS context TripType
item 1382176 cmax is inf con FAMILY context TripType


In [7]:
import os
#should be the same path in Jupyter where this notebook exists
notebook_path = os.path.abspath("CA_NCF_tripadvisor.ipynb")

In [8]:
sqlContext = SQLContext(sc)
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import col, udf

#reading raw data from CSV into a DataFrame

Rating = Row("UserID", "ItemID", "Rating","UserState","UserTimeZone","ItemCity","ItemState","ItemTimeZone","TripType")

ratingsget = os.path.join(os.path.dirname(notebook_path), "datasets/tripAdvisor/Data_TripAdvisor_v2.csv")
ratings= sc.textFile(ratingsget) \
    .map(lambda line: line.split(",")[0:9])\
    .map(lambda line: (line[0], int(line[1]), int(line[2]), line[3],line[4],line[5],line[6],line[7],line[8]))\
    .map(lambda r: Rating(*r))
ratingDF_OLD_VERY = sqlContext.createDataFrame(ratings)
ratingDF_OLD_VERY.show(5)

someDFtt = ratingDF_OLD_VERY.select(col("UserID")).distinct()

#max_movie_id_alt = someDFt.count()

print("UserID :   " + str(someDFtt.count()))

someDFtt1 = ratingDF_OLD_VERY.select(col("ItemID")).distinct()

#max_movie_id_alt = someDFt.count()

print("ItemID :   " + str(someDFtt1.count()))


+--------------------+------+------+---------+------------+----------+---------+------------+--------+
|              UserID|ItemID|Rating|UserState|UserTimeZone|  ItemCity|ItemState|ItemTimeZone|TripType|
+--------------------+------+------+---------+------------+----------+---------+------------+--------+
|5C28F393B23BB8945...|219668|     5|       AK|          AK|GREENSBORO|       NC|     EASTERN|    SOLO|
|3FA27F6E8AC712A82...|223860|     5|       AK|          AK|   PHOENIX|       AZ|    MOUNTAIN|    SOLO|
|B99CFBB5411EDC888...| 75680|     5|       AK|          AK|   ANAHEIM|       CA|     PACIFIC|  FAMILY|
|3FA27F6E8AC712A82...|224783|     5|       AK|          AK|   SEATTLE|       WA|     PACIFIC|    SOLO|
|7CEFF5C32BA1F3B18...|222984|     5|       AK|          AK|     MIAMI|       MI|     EASTERN| COUPLES|
+--------------------+------+------+---------+------------+----------+---------+------------+--------+
only showing top 5 rows

UserID :   2371
ItemID :   2269


In [ ]:
####all the following cells are preparations for incorporating context into original items
####we are actual constructing first new artificial items set
####then we apply the NCF form the library as-is

In [9]:
populateList = []
def condition(i):
    #note the permutation for each possible context condition, this will be used for binarization hereafter
    switcher={
            'COUPLES':["COUPLES", "SOLO", "FAMILY", "FRIENDS" ,"BUSINESS"],
            'SOLO':["SOLO", "COUPLES", "FAMILY", "FRIENDS" ,"BUSINESS"],
            'FAMILY':["FAMILY", "SOLO", "COUPLES", "FRIENDS" ,"BUSINESS"],
            'FRIENDS':["FRIENDS", "SOLO", "FAMILY", "COUPLES" ,"BUSINESS"],
            'BUSINESS':["BUSINESS", "SOLO", "FAMILY", "COUPLES" ,"FRIENDS"],
         
           
         }
    return switcher.get(i,"Invalid year")

In [10]:
from pyspark.sql.functions import col, udf
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Binarizer
from pyspark.sql.functions import rand 
#transformations
indexer = StringIndexer(inputCol="UserID", outputCol="newUserID1")
indexer1 = StringIndexer(inputCol="ItemID", outputCol="indexedItemID2")
indexer2 = StringIndexer(inputCol="artItemID1", outputCol="artItemID2")
binarizer22 = Binarizer(threshold=1.0, inputCol="locationIndexed", outputCol="binarized_location")

In [11]:
#transformations: applying indexers to ratingDF_OLD_VERY in a pipeline (one after the other)
pipeline = Pipeline(stages=[indexer,indexer1])
ratingDF_OLD = pipeline.fit(ratingDF_OLD_VERY).transform(ratingDF_OLD_VERY)

In [12]:
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, udf
df_list = []
#preparing for the application of itemSplitting procedure above by separating items to-be-split from the original list
toDiscard_df_list = []

#remember that some_list contains split items
for obj in some_list:
    itemid= obj.item
    ctxCond = obj.condition
    ctx = obj.context
    partialDF = ratingDF_OLD.select("*").where((col("ItemID")==itemid) )
    
    #toDiscard_df_list is a list of DFs, will be union later on
    toDiscard_df_list.append(partialDF)
    #print(partialDF.count())
    # applying the switcher above to check based on what we split the items (binarization is the key here)
    populateList = condition(ctxCond)
    print(populateList)
    print(ctx)
    #a user defined function
    general_udf = udf(lambda somename: categorical_from_vocab_list(somename, populateList, start=1))
    partialDF_indexed = partialDF.withColumn("locationIndexed", general_udf(col(ctx)).cast("double"))
    #print(partialDF_indexed.where(col('locationIndexed') != 0.0).count())
    #print(partialDF_indexed.count())
    pipeline1 = Pipeline(stages = [binarizer22])
    allDF_temp =  pipeline1.fit(partialDF_indexed).transform(partialDF_indexed)
    allDF_temp.select('UserID','newUserID1','ItemID','binarized_location','TripType','indexedItemID2').show(5)

    bucket_cross_udf = udf(lambda feature1, feature2: hash_bucket(str(feature1) + "_" + str(feature2), bucket_size=100))
    df_list.append(allDF_temp.withColumn("artItemID1", bucket_cross_udf(col("indexedItemID2"), col("binarized_location")).cast("int") + 1))

['SOLO', 'COUPLES', 'FAMILY', 'FRIENDS', 'BUSINESS']
TripType
+--------------------+----------+------+------------------+--------+--------------+
|              UserID|newUserID1|ItemID|binarized_location|TripType|indexedItemID2|
+--------------------+----------+------+------------------+--------+--------------+
|20A4EFC475E6F8B29...|    2230.0|503601|               0.0|    SOLO|         619.0|
|C5FB378CCB9FB9FFF...|    1829.0|503601|               0.0|    SOLO|         619.0|
|718748B9BAAEB7915...|      93.0|503601|               0.0|    SOLO|         619.0|
|05D57A581E9DAE14F...|      17.0|503601|               1.0|BUSINESS|         619.0|
|B95E2F08E14A7A45E...|    2220.0|503601|               1.0|BUSINESS|         619.0|
+--------------------+----------+------+------------------+--------+--------------+
only showing top 5 rows

['SOLO', 'COUPLES', 'FAMILY', 'FRIENDS', 'BUSINESS']
TripType
+--------------------+----------+------+------------------+--------+--------------+
|          

In [13]:
allDF_prepare = reduce(DataFrame.unionAll, df_list)#.cache()


In [14]:
allDF_prepare.show(1)

+--------------------+------+------+---------+------------+--------+---------+------------+--------+----------+--------------+---------------+------------------+----------+
|              UserID|ItemID|Rating|UserState|UserTimeZone|ItemCity|ItemState|ItemTimeZone|TripType|newUserID1|indexedItemID2|locationIndexed|binarized_location|artItemID1|
+--------------------+------+------+---------+------------+--------+---------+------------+--------+----------+--------------+---------------+------------------+----------+
|20A4EFC475E6F8B29...|503601|     5|       CA|     PACIFIC|LASVEGAS|       NV|     PACIFIC|    SOLO|    2230.0|         619.0|            1.0|               0.0|        90|
+--------------------+------+------+---------+------------+--------+---------+------------+--------+----------+--------------+---------------+------------------+----------+
only showing top 1 row



In [15]:
from pyspark.sql.functions import lit
#discarding all partial DFs in the toDiscard_df_list constructed beforehand
toDiscard_df_all = reduce(DataFrame.unionAll, toDiscard_df_list)#.cache()
#print(toDiscard_df_all.select(col('artItemID1')).distinct().count())
#toDiscard_df_all_sub now contains all items to split
toDiscard_df_all_sub = toDiscard_df_all.select("*")
ratingDF_OLD_sub = ratingDF_OLD#.withColumn("indexedItemID2",col("ItemID"))
#extract all items except split items
remainingDF = ratingDF_OLD_sub.subtract(toDiscard_df_all_sub)

remainingDF_indexed = remainingDF.withColumn('binarized_location',lit(1.0))
bucket_cross_udf1 = udf(lambda feature1, feature2: hash_bucket(str(feature1) + "_" + str(feature2), bucket_size=1200000))
remainingDF_indexed_all = remainingDF_indexed.withColumn("artItemID1", bucket_cross_udf1(col("ItemID"), col("binarized_location")).cast("int") + 1)

In [16]:
#transformations
remainingDF_indexed_all_sub = remainingDF_indexed_all.select(col('newUserID1') ,col('indexedItemID2'),col("UserID"),
                                        col('Rating'),col('ItemID'),col('artItemID1'),col("binarized_location"))
allDF_prepare_sub_sub = allDF_prepare.select(col("newUserID1") ,col("indexedItemID2"),col("UserID"),
                                        col("Rating"),col("ItemID"),col("artItemID1"),col("binarized_location"))
allDF_difinite = remainingDF_indexed_all_sub.union(allDF_prepare_sub_sub)#.cache()

In [17]:
someDF_art = allDF_difinite.select(col("artItemID1")).distinct()
#print(someDF_art.count())

In [18]:
pipeline2 = Pipeline(stages = [indexer2])

allDF =  pipeline2.fit(allDF_difinite).transform(allDF_difinite)

someDF1 = allDF.select((col("newUserID1").cast("int") + 1).alias("newUserID"),col("indexedItemID2").cast("int"),
                       col("Rating"),col("ItemID"),(col("artItemID2").cast("int") + 1).alias("aritid"))
someDF2 = allDF.select((col("newUserID1").cast("int") + 1).alias("newUserID"),
                       (col("artItemID2").cast("int") + 1).alias("artItemID"),col("Rating"))#.distinct()

someDF_OLD = someDF2.orderBy(rand())#.select("*").limit(5000000)



In [19]:
#  discard items with less than 20 ratings (interactions)
import pyspark.sql.functions as f
from pyspark.sql import Window
w = Window.partitionBy('artItemID')
someDF_NEW = someDF_OLD.withColumn('count_artItemID', f.count('newUserID').over(w))\
    .where(f.col('count_artItemID') > 5)\
    .drop('count_artItemID')#.select("*").limit(5000000)


In [20]:
w = Window.partitionBy('newUserID')

#the final dataFrame that contains the new list with items split
someDF = someDF_NEW.withColumn('count_newUserID', f.count('artItemID').over(w))\
.where(f.col('count_newUserID') > 1).drop('count_newUserID')

In [ ]:
####now that we have constructed artificial items set, we are then ready to apply NCF from BigDL

comprehensing your data. 
a tuple is (userid, movieid, rating). 

In [21]:
#we need this information to feed it to the NCF model in bigDL
someDFt3 = someDF.select(col("newUserID")).distinct()

max_user_id1 =  someDF.agg({"newUserID": "max"}).collect()[0][0]
print("max newUserID " + str(someDF.agg({"newUserID": "max"}).collect()[0][0]))

someDFt4 = someDF.select(col("artItemID")).distinct()
print("max artItemID " + str(someDF.agg({"artItemID": "max"}).collect()[0][0]))
max_item_id1 = someDF.agg({"artItemID": "max"}).collect()[0][0]
print(max_user_id1, max_item_id1)

max newUserID 2371
max artItemID 747
2371 747


Transforming raw rarings into an RDD . BigDl 6.0 requires input as RDD of samples, 
BigDL data structure that can be formed by two numpy arrays containing features and lables for the first and 
second array, respectively. 
in the following format Sample.from_ndarray(feature, label)

In [22]:

def build_sample(user_id, item_id, rating):
    sample = Sample.from_ndarray(np.array([user_id, item_id]), np.array([rating]))
    return UserItemFeature(user_id, item_id, sample)
#CA-NCF settings
pairFeatureRdds_CANCF = someDF.rdd.map(lambda x: build_sample(x[0], x[1],x[2]))

pairFeatureRdds_CANCF.repartition(1)

pairFeatureRdds_CANCF.take(3)

In [23]:
#see more parameters here:
#https://github.com/intel-analytics/analytics-zoo/blob/master/docs/docs/APIGuide/Models/recommendation.md
#extra hyperparameters , user_embed=20, item_embed=20
ncf = NeuralCF(user_count=max_user_id1, item_count=max_item_id1, class_num=5, hidden_layers=[40,20])#, include_mf = True,mf_embed = 10)

creating: createZooNeuralCF


In [24]:
import time
millis = int(round(time.time() * 1000))
##80% training data, 20% testing, randomly split
#CA-NCF settings
trainPairFeatureRdds_CANCF, valPairFeatureRdds_CANCF = pairFeatureRdds_CANCF.randomSplit([0.8, 0.2], seed = millis)
valPairFeatureRdds_CANCF.cache()
train_rdd_CANCF= trainPairFeatureRdds_CANCF.map(lambda pair_feature: pair_feature.sample)
val_rdd_CANCF= valPairFeatureRdds_CANCF.map(lambda pair_feature: pair_feature.sample)

In [25]:
##predicting user-item pairs using CA-NCF
predictUserItem_CANCF = ncf.predict_user_item_pair(valPairFeatureRdds_CANCF)
for ans in predictUserItem_CANCF.take(5): print(ans)

UserItemPrediction [user_id: 148, item_id: 192, prediction: 5, probability: 0.20879336842252322]
UserItemPrediction [user_id: 148, item_id: 422, prediction: 2, probability: 0.20849703814530976]
UserItemPrediction [user_id: 833, item_id: 353, prediction: 5, probability: 0.20615691935021]
UserItemPrediction [user_id: 1238, item_id: 83, prediction: 2, probability: 0.20535563770586102]
UserItemPrediction [user_id: 1580, item_id: 578, prediction: 2, probability: 0.20632552923607234]


In [26]:
#Recommending 5 items for every user
recItemsForUsers_CANCF = ncf.recommend_for_user(valPairFeatureRdds_CANCF, 5)
for ans in recItemsForUsers_CANCF.take(10): print(ans)

UserItemPrediction [user_id: 1200, item_id: 716, prediction: 5, probability: 0.2096270549772194]
UserItemPrediction [user_id: 2000, item_id: 164, prediction: 5, probability: 0.2079028628742416]
UserItemPrediction [user_id: 600, item_id: 162, prediction: 2, probability: 0.20755526144083172]
UserItemPrediction [user_id: 600, item_id: 410, prediction: 2, probability: 0.20653570996804926]
UserItemPrediction [user_id: 400, item_id: 462, prediction: 5, probability: 0.2071276076240727]
UserItemPrediction [user_id: 400, item_id: 476, prediction: 5, probability: 0.20706403667906573]
UserItemPrediction [user_id: 401, item_id: 287, prediction: 2, probability: 0.20675529799137107]
UserItemPrediction [user_id: 201, item_id: 734, prediction: 5, probability: 0.20874777472144673]
UserItemPrediction [user_id: 201, item_id: 337, prediction: 5, probability: 0.20851518292697024]
UserItemPrediction [user_id: 201, item_id: 18, prediction: 5, probability: 0.20807441378078098]
